# Data Exploration, Visualization and Processing. Model Development and Deployment

This notebook is a one-stop-shop for replicating the project results. It will serve as the project development environment. Please refer to the `README` to get a background on the project, to understand its objectives. 

The image below gives a high level architecture of the various steps involved in this notebook:

### High level Process Flow
<img src=Project_Workflow.png width=90% />

To run this notebook, follow these instructions:
- git clone the repository at https://github.com/pharouhk/Querying-and-forecasting-with-Natural-Language.git to quickly pull all the project dependencies
- make sure that your Python environment has both TensorFlow and PyTorch installed
- It is important to also note this notebook was developed using AWS Sagemaker, so an AWS profile and access to compute instances is required